In [2]:
# Cell 1: Setup and Imports
# Uncomment the line below if you need to install the libraries
# !pip install pandas numpy plotly

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully.")

Libraries imported successfully.


In [3]:
# Cell 2: Feature Definitions and Load Data Function (The FIX)

# --- Feature Definitions ---
NUMERICAL_FEATURES = ['age','duration','campaign','pdays','previous',
                      'emp_var_rate','cons_price_idx','cons_conf_idx','euribor3m','nr_employed']
CATEGORICAL_FEATURES = [
    'job', 'marital', 'education', 'default', 'housing', 
    'loan', 'contact', 'month', 'day_of_week', 'poutcome'
]


In [4]:
# Cell 3: Load Data and Check (Final FIX for Absolute Path)
import os

# --- ACTION REQUIRED: PASTE YOUR ABSOLUTE PATH HERE ---
# Use the 'r' prefix to handle backslashes correctly in the path
absolute_file_path = r"E:\IBA_MS_DS 2026\Unsupervised_Machine_Learning\Project\streamlit_app\bank-additional\bank-additional-full.xlsx"
# -------------------------------------------------------------

def load_absolute_data(path, numerical_features, categorical_features):
    """Loads the data directly from the absolute path and applies the required cleaning."""
    df = pd.DataFrame()
    print(f"Attempting to load data from absolute path: {path}")
    
    try:
        # Load the XLSX file directly
        df = pd.read_excel(path)
        
        if df.empty:
            raise Exception("File loaded but returned an empty DataFrame.")
            
        # 1. Clean column names (lower, strip, replace '.' with '_')
        df.columns = df.columns.str.strip().str.lower().str.replace('.', '_', regex=False)
        
        # 2. Numerical handling (Impute with Median and convert to numeric)
        for c in numerical_features:
            if c in df.columns:
                df[c] = pd.to_numeric(df[c], errors='coerce')
                if df[c].isnull().any():
                    df[c] = df[c].fillna(df[c].median())
        
        # 3. Categorical handling (Impute with Mode/Unknown and standardize)
        for c in categorical_features:
            if c in df.columns:
                df[c] = df[c].astype(str).str.strip().str.lower()
                if df[c].isnull().any() or (df[c] == 'nan').any():
                    mode_val = df[c].mode()
                    mode_fill = mode_val[0] if len(mode_val) > 0 else 'unknown'
                    df[c] = df[c].replace('nan', mode_fill).fillna(mode_fill)

        # 4. Create binary target
        if 'y' in df.columns:
            df['y_binary'] = (df['y'] == 'yes').astype(int)
        
        print(f"✅ Data successfully loaded and cleaned.")
        return df

    except FileNotFoundError:
        print(f"❌ ERROR: File not found at the specified path: {path}")
        return pd.DataFrame()
    except Exception as e:
        print(f"❌ ERROR loading or cleaning data: {e}")
        return pd.DataFrame()


# IMPORTANT: This assumes NUMERICAL_FEATURES and CATEGORICAL_FEATURES 
# from Cell 2 are defined and accessible.
try:
    df_original = load_absolute_data(absolute_file_path, NUMERICAL_FEATURES, CATEGORICAL_FEATURES)
except NameError:
    print("❌ NameError: Please ensure Cell 2 defining NUMERICAL_FEATURES and CATEGORICAL_FEATURES was run first.")
    df_original = pd.DataFrame()


if df_original.empty:
    raise ValueError("Data loading failed. Check the absolute path and ensure the file is not corrupted.")

# Use the original data for all steps in this notebook
df_filtered = df_original.copy()

print(f"\nData successfully loaded. Shape: {df_filtered.shape}")
print("\nFirst 5 rows of the cleaned data:")
print(df_filtered[['age', 'campaign', 'y', 'y_binary']].head())

Attempting to load data from absolute path: E:\IBA_MS_DS 2026\Unsupervised_Machine_Learning\Project\streamlit_app\bank-additional\bank-additional-full.xlsx
✅ Data successfully loaded and cleaned.

Data successfully loaded. Shape: (41188, 22)

First 5 rows of the cleaned data:
   age  campaign   y  y_binary
0   56         1  no         0
1   57         1  no         0
2   37         1  no         0
3   40         1  no         0
4   56         1  no         0


In [5]:
# Cell 4: Target Variable Distribution

print("\n--- Target Variable Distribution ---")
target_counts = df_filtered['y'].value_counts()
yes_count = int(target_counts.get('yes', 0))
no_count = int(target_counts.get('no', 0))
total = yes_count + no_count
yes_pct = (yes_count / total * 100) if total > 0 else 0

print(f"Total Subscriptions (Yes): {yes_count:,}")
print(f"Conversion Rate: {yes_pct:.2f}%")

fig_target = px.pie(
    names=['No Subscription', 'Subscription'],
    values=[no_count, yes_count],
    color_discrete_sequence=['#ff7f0e', '#2ca02c'],
    hole=0.5,
    title=f"Target Variable Distribution (Total: {total:,} customers)"
)
fig_target.show()


--- Target Variable Distribution ---
Total Subscriptions (Yes): 4,640
Conversion Rate: 11.27%


In [6]:
# Cell 5: Numerical Feature Analysis

print("\n--- Numerical Feature Analysis ---")
selected_num = 'duration' # Example: Use 'duration'

if selected_num in df_filtered.columns:
    print(f"\nStatistics for '{selected_num}': Mean={df_filtered[selected_num].mean():.2f}, Median={df_filtered[selected_num].median():.2f}")
    
    # Distribution/Histogram
    fig_hist = px.histogram(
        df_filtered,
        x=selected_num,
        color='y',
        title=f"Distribution of {selected_num} by Subscription Status",
        nbins=40,
        color_discrete_map={'no': '#ff7f0e', 'yes': '#2ca02c'},
        opacity=0.7,
        barmode='overlay',
        marginal='box'
    )
    fig_hist.update_layout(height=450)
    fig_hist.show()
    
    # Boxplot by Target
    fig_box = px.box(
        df_filtered,
        x='y',
        y=selected_num,
        color='y',
        title=f"{selected_num} Distribution by Subscription Status",
        color_discrete_map={'no': '#ff7f0e', 'yes': '#2ca02c'},
        points='outliers'
    )
    fig_box.update_layout(height=450, showlegend=False)
    fig_box.show()


--- Numerical Feature Analysis ---

Statistics for 'duration': Mean=258.29, Median=180.00


In [7]:
# Cell 6: Categorical Feature Analysis

print("\n--- Categorical Feature Analysis ---")
selected_cat = 'job' 

if selected_cat in df_filtered.columns:
    # Frequency Distribution
    counts = df_filtered[selected_cat].value_counts().head(10)
    fig_bar = px.bar(
        x=counts.index.astype(str),
        y=counts.values,
        title=f"Top 10 Categories in '{selected_cat}' (Frequency Distribution)",
        labels={'x': selected_cat, 'y': 'Count'},
        text=[f"{v:,}" for v in counts.values],
        color=counts.values,
        color_continuous_scale='Blues'
    )
    fig_bar.update_layout(height=450, showlegend=False)
    fig_bar.show()

    # Conversion Rate (Combo Chart)
    conv_rates_series = df_filtered.groupby(selected_cat)['y_binary'].mean() * 100
    total_counts_series = df_filtered[selected_cat].value_counts()
    
    conv_df = pd.DataFrame({
        'category': conv_rates_series.index,
        'conv_rate': conv_rates_series.values,
        'total_count': total_counts_series.reindex(conv_rates_series.index).fillna(0).astype(int)
    }).sort_values('conv_rate', ascending=False).head(10).reset_index(drop=True)
    
    fig_conv = go.Figure()
    
    # Bar for Total Contacts (Secondary Y-axis)
    fig_conv.add_trace(go.Bar(
        x=conv_df['category'].astype(str),
        y=conv_df['total_count'],
        name='Total Contacts',
        marker_color='lightblue',
        yaxis='y2'
    ))

    # Line/Scatter for Conversion Rate (Primary Y-axis)
    fig_conv.add_trace(go.Scatter(
        x=conv_df['category'].astype(str),
        y=conv_df['conv_rate'],
        name='Conversion Rate (%)',
        mode='lines+markers+text',
        line=dict(width=3, color='red'),
        marker=dict(size=10),
        text=[f"{rate:.2f}%" for rate in conv_df['conv_rate']],
        textposition='top center',
        yaxis='y'
    ))
    
    fig_conv.update_layout(
        title=f"Conversion Rate by Top Categories in '{selected_cat}'",
        xaxis=dict(title=selected_cat),
        yaxis=dict(title='Conversion Rate (%)', side='left'),
        yaxis2=dict(title='Total Contacts', side='right', overlaying='y', showgrid=False),
        height=450,
        hovermode='x unified',
        showlegend=True
    )
    fig_conv.show()


--- Categorical Feature Analysis ---


In [8]:
# Cell 7: Correlation Analysis (Heatmap)

print("\n--- Correlation Analysis ---")

# Ensure only existing numerical columns + target are used
corr_cols = [col for col in NUMERICAL_FEATURES if col in df_filtered.columns] + ['y_binary']
num_df = df_filtered[corr_cols].copy()
corr = num_df.corr(numeric_only=True)

print("Correlation Matrix Head:")
print(corr.head())

# Strongest Correlations with Target
if 'y_binary' in corr.columns:
    target_corr = corr['y_binary'].drop('y_binary', errors='ignore')
    target_corr_sorted = target_corr.abs().sort_values(ascending=False).head(8)
    
    print("\nStrongest Correlations with Target:")
    for feat in target_corr_sorted.index:
        val = corr.loc[feat, 'y_binary']
        print(f"{'🟢' if val > 0 else '🔴'} {feat}: {val:+.4f}")

# Feature Correlation Heatmap
if len(corr.columns) > 1:
    fig_corr = go.Figure(data=go.Heatmap(
        z=corr.values,
        x=corr.columns,
        y=corr.columns,
        colorscale='RdBu',
        zmid=0,
        zmin=-1,
        zmax=1,
        text=np.round(corr.values, 3),
        texttemplate='%{text}',
        textfont={"size": 9},
        colorbar=dict(title="Correlation")
    ))
    
    fig_corr.update_layout(
        title="Feature Correlation Heatmap",
        height=600,
        xaxis={'side': 'bottom', 'tickangle': 45},
        yaxis={'autorange': 'reversed'}
    )
    fig_corr.show()
else:
    print("Not enough numerical features to display a correlation heatmap.")


--- Correlation Analysis ---
Correlation Matrix Head:
               age  duration  campaign     pdays  previous  emp_var_rate  \
age       1.000000 -0.000866  0.004594 -0.034369  0.024365     -0.000371   
duration -0.000866  1.000000 -0.071699 -0.047577  0.020640     -0.027968   
campaign  0.004594 -0.071699  1.000000  0.052584 -0.079141      0.150754   
pdays    -0.034369 -0.047577  0.052584  1.000000 -0.587514      0.271004   
previous  0.024365  0.020640 -0.079141 -0.587514  1.000000     -0.420489   

          cons_price_idx  cons_conf_idx  euribor3m  nr_employed  y_binary  
age             0.000857       0.129372   0.010767    -0.017725  0.030399  
duration        0.005312      -0.008173  -0.032897    -0.044703  0.405274  
campaign        0.127836      -0.013733   0.135133     0.144095 -0.066357  
pdays           0.078889      -0.091342   0.296899     0.372605 -0.324914  
previous       -0.203130      -0.050936  -0.454494    -0.501333  0.230181  

Strongest Correlations with Tar